In [1]:
import pandas as pd
import requests
import time
import re
import unicodedata
import json
from datetime import datetime

In [78]:
df_stations_info =  pd.read_csv('stations_info_history.csv')

In [79]:
df_stations_info.head()

,city,station_name,station_id,station_lat,station_lon,station_url,history_file
0,Jelenia Góra,"Jelenia Góra - Ogińskiego, Poland",8135,50.913470,15.765659,poland/dolnoslaskie/jelenia-gora-oginskiego,jelenia-góra - ogińskiego-air-quality.csv
1,Biała Podlaska,"Biała Podlaska, Poland",6479,52.038713,23.144503,poland/lublin/biala-podlaska,biała-podlaska-air-quality.csv
2,Białystok,"Białystok-Miejska, Poland",6520,53.126815,23.156370,poland/bialystok/bialystok-miejska,białystok-miejska-air-quality.csv
3,Bielsko-Biała,"Bielsko-Biała, ul. Partyzantów, Poland",10544,49.802089,19.048629,poland/slaski/bielsko-biala--ul.-partyzantow,"bielsko-biała,-ul. partyzantów-air-quality.csv"
4,Bielsko-Biała,"Bielsko-Biała, ul. Kossak-Szczuckiej 19, Poland",6535,49.813464,19.027318,poland/slaski/bielsko-biala--ul.-kossak-szczuc...,"bielsko-biała,-ul. kossak-szczuckiej 19-air-qu..."


In [115]:
history_folder = 'measures_history'

In [290]:
history_measurements_df = pd.DataFrame(columns=['year', 'month', 'city', 'station_name', 'station_lat', 'station_lon', 
                                                'pm25', 'pm10', 'o3', 'no2', 'so2', 'co'])

In [291]:
def history_processing(current_row, current_df):
    station_history_df['date'] = pd.to_datetime(station_history_df['date'], format='%Y/%m/%d')
    station_history_df['date'] = pd.to_datetime(station_history_df['date'].dt.strftime('%Y-%m-%d'))
    station_history_df_sorted = station_history_df.sort_values(by='date', ascending=False)
    to_float_columns = station_history_df_sorted.columns.difference(['date'])
    station_history_df_sorted[to_float_columns] = station_history_df_sorted[to_float_columns].applymap(lambda x: pd.to_numeric(x, errors='coerce'))
    station_history_df_sorted['year'] = station_history_df_sorted['date'].dt.year
    station_history_df_sorted['month'] = station_history_df_sorted['date'].dt.month
    monthly_avg = station_history_df_sorted.groupby(['year', 'month']).mean().reset_index()
    result_df = pd.DataFrame()
    num_cols = [col for col in monthly_avg.columns if col not in ['date']]
    for col in num_cols:
        result_df[col] = monthly_avg[col]
    result_cols = result_df.columns
    result_df_sorted = result_df.sort_values(by=['year', 'month'], ascending=False)
    for res_index, res_row in result_df_sorted.iterrows():         
        params = {'pm25': None, 'pm10': None, 'o3': None, 'no2': None, 'so2': None, 'co': None}
        for res_col in result_cols:
            if res_col!='year' and res_col!='month':
                params[res_col.strip()] = res_row[res_col]
        history_measurements_df.loc[len(history_measurements_df)] = [int(res_row['year']), int(res_row['month']), row['city'], row['station_name'], row['station_lat'], row['station_lon'],
        params['pm25'], params['pm10'], params['o3'], params['no2'], params['so2'], params['co']]

In [292]:
for index, row in df_stations_info.iterrows():
    print(f'Przetwarzanie historii stacji: {index+1}')
    try:
        file_name = row['history_file']
        station_history_df = pd.read_csv(f'{history_folder}/{file_name}')
        history_processing(row, station_history_df)
    except Exception:
        print("\tNie udało się pobrać pliku z historią.")

Przetwarzanie historii stacji: 1
Przetwarzanie historii stacji: 2
Przetwarzanie historii stacji: 3
Przetwarzanie historii stacji: 4
Przetwarzanie historii stacji: 5
Przetwarzanie historii stacji: 6
Przetwarzanie historii stacji: 7
Przetwarzanie historii stacji: 8
Przetwarzanie historii stacji: 9
Przetwarzanie historii stacji: 10
Przetwarzanie historii stacji: 11
Przetwarzanie historii stacji: 12
Przetwarzanie historii stacji: 13
Przetwarzanie historii stacji: 14
Przetwarzanie historii stacji: 15
Przetwarzanie historii stacji: 16
Przetwarzanie historii stacji: 17
Przetwarzanie historii stacji: 18
Przetwarzanie historii stacji: 19
Przetwarzanie historii stacji: 20
Przetwarzanie historii stacji: 21
Przetwarzanie historii stacji: 22
Przetwarzanie historii stacji: 23
Przetwarzanie historii stacji: 24
Przetwarzanie historii stacji: 25
Przetwarzanie historii stacji: 26
Przetwarzanie historii stacji: 27
Przetwarzanie historii stacji: 28
Przetwarzanie historii stacji: 29
Przetwarzanie historii 

In [301]:
history_measurements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10442 entries, 0 to 10441
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          10442 non-null  int64  
 1   month         10442 non-null  int64  
 2   city          10442 non-null  object 
 3   station_name  10442 non-null  object 
 4   station_lat   10442 non-null  float64
 5   station_lon   10442 non-null  float64
 6   pm25          5184 non-null   float64
 7   pm10          9591 non-null   float64
 8   o3            6343 non-null   float64
 9   no2           9327 non-null   float64
 10  so2           7860 non-null   float64
 11  co            4824 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 1.0+ MB


In [302]:
history_measurements_df_final = history_measurements_df.dropna(subset=['pm25', 'pm10', 'o3', 'no2', 'so2', 'co'], how='all')

In [303]:
history_measurements_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10442 entries, 0 to 10441
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          10442 non-null  int64  
 1   month         10442 non-null  int64  
 2   city          10442 non-null  object 
 3   station_name  10442 non-null  object 
 4   station_lat   10442 non-null  float64
 5   station_lon   10442 non-null  float64
 6   pm25          5184 non-null   float64
 7   pm10          9591 non-null   float64
 8   o3            6343 non-null   float64
 9   no2           9327 non-null   float64
 10  so2           7860 non-null   float64
 11  co            4824 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 1.0+ MB


In [305]:
history_measurements_df_final.to_csv('collected_data/average_monthly_historical_measurements.csv', index=False)